# FAKE NEWS PREDICTION USING MACHINE LEARNING
## CHARLES DARWIN UNIVERSITY | COLLEGE OF ENGINEERING, INFORMATION TECHNOLOGY AND ENVIRONMENT
### Ananda Tiwari (Student ID: S333652)


### Library Import


In [1]:
import pandas as pnda
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

### Import and prepare Dataset

In [2]:
fake_dataset = pnda.read_csv("Compiled_Fake.csv")
true_dataset = pnda.read_csv("Compiled_True.csv")

Categorising fake as 0 and true as 1 and adding column class in the dataset

In [3]:
fake_dataset["class"] = 0
true_dataset["class"] = 1

Taking out 5 data from both fake and true dataset for manual testing 

In [4]:
fake_dataset.shape, true_dataset.shape

((25, 14), (96, 15))

In [5]:
fake_dataset_manual_testing = fake_dataset.tail(5)
for i in range(24,19,-1):
    fake_dataset.drop([i], axis = 0, inplace = True)
true_dataset_manual_testing = true_dataset.tail(5)
for i in range(95,90,-1):
    true_dataset.drop([i], axis = 0, inplace = True)

In [6]:
fake_dataset.shape, true_dataset.shape

((20, 14), (91, 15))

combining manual testing datasets and save in a single file

In [7]:
fake_dataset_manual_testing["class"] = 0
true_dataset_manual_testing["class"] = 1

C:\Users\Ananda\AppData\Local\Temp/ipykernel_996/2564063938.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fake_dataset_manual_testing["class"] = 0
C:\Users\Ananda\AppData\Local\Temp/ipykernel_996/2564063938.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_dataset_manual_testing["class"] = 1


In [8]:
fake_dataset_manual_testing.head(10)

,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count,Title,Content,class
20,1.840000e+14,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/27/2016,link,False,yes,NaN,93,68,BREAKING: The First Charges Have Been Filed I...,Special Counsel Robert Mueller appears to have...,0
21,6.231759e+10,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/20/2016,link,False,NaN,159.0,429,49,Trump And John Kelly Just Got RIPPED APART By...,Donald Trump and his disgraceful team deserve ...,0
22,6.231759e+10,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/22/2016,link,False,NaN,15.0,181,70,NRA Releases Fascist Ad Calling All Trump Cri...,The First Amendment of the United States Const...,0
23,6.231759e+10,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/26/2016,link,False,NaN,36.0,189,27,"Trump Takes Credit For Releasing JFK Files, B...","Saturday morning, Trump was going through his ...",0
24,6.231759e+10,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,False,NaN,222.0,1043,39,Bill Maher Takes Down Trumpâ€™s Lying Chief O...,White House Chief of Staff John Kelly is a Fo...,0


In [9]:
true_dataset_manual_testing.head(10)

,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count,Title,Content,class
91,6.231759e+10,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,True,yes,16,203,37,First Trump-Clinton debate is the most-watched...,The first general election debate between Hill...,1
92,6.231759e+10,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,True,NaN,1286,8025,238,The Arizona Republic endorses Hillary Clinton,"For the first time in its 126-year history, Th...",1
93,6.231759e+10,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,True,NaN,94,385,32,"Shimon Peres, Israeli leader for 8 decades, di...","Shimon Peres, whose public career spanned the ...",1
94,6.231759e+10,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,True,NaN,1127,6443,201,Former Va. Sen. John Warner slams Trump during...,Former Republican Sen. John Warner endorsed Hi...,1
95,6.231759e+10,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/27/2016,link,True,NaN,190,1974,408,Chelsea Clinton: Father’s infidelity talk just...,Chelsea Clinton is in no rush to thank Donald ...,1


In [10]:
df_manual_testing = pnda.concat([fake_dataset_manual_testing,true_dataset_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

Combining true and fake news dataframe together

In [11]:
df_marge = pnda.concat([fake_dataset, true_dataset], axis =0 )
df_marge.head(10) 

,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count,Title,Content,class,account_id
0,1.040000e+15,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/19/2016,link,False,NaN,25.0,207,108,Fatina Abdrabboh: Islam 'Has Deep Roots' in US,"Fatina Abdrabboh, Director of American Arab An...",0,NaN
1,1.040000e+15,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/22/2016,link,False,NaN,38.0,138,32,President Obama Shares Emotional Video of Chil...,As criticism continues over Donald Trump Jr.’s...,0,NaN
2,1.040000e+15,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/26/2016,link,False,NaN,205.0,1049,997,5 Storylines to Watch During Tonight's Preside...,As millions of voters tune in to watch the fir...,0,NaN
3,1.040000e+15,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/26/2016,link,False,NaN,2.0,50,34,Stars Comment on Social Media Before Tonight's...,Tonight's presidential debate between Hillary ...,0,NaN
4,1.040000e+15,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/26/2016,link,False,yes,228.0,447,147,ANALYSIS: Hillary Clinton Baits Donald Trump I...,Hillary Clinton used the first presidential de...,0,NaN
5,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/23/2016,link,False,NaN,61.0,608,92,Robert Kennedy Would Have Hated Donald Trump,writing as a former legislative assistant to S...,0,NaN
6,1.020000e+16,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,9/23/2016,link,False,NaN,5.0,69,47,Nerdcast 2016: Are you ready for debate night?,"It’s time for Episode 21 of the 2016 Nerdcast,...",0,NaN
7,1.840000e+14,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/19/2016,link,False,NaN,3.0,9,3,Guilty As Hell Paul Manafort And Rick Gates E...,The world is exploding right now due to the ne...,0,NaN
8,1.840000e+14,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/19/2016,link,False,NaN,NaN,24,7,This Is What Fox News Considered â€˜Newsâ€™ A...,"This morning, Trump s former campaign manager ...",0,NaN
9,1.840000e+14,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,9/19/2016,link,False,NaN,5.0,19,6,Racistsâ€™ Heads Will Explode When They Find ...,Despite what many racists say and regardless o...,0,NaN


In [12]:
df_marge.columns

Index(['post_id', 'Category', 'Page', 'Post URL', 'Date Published',
       'Post Type', 'Rating', 'Debate', 'share_count', 'reaction_count',
       'comment_count', 'Title', 'Content', 'class', 'account_id'],
      dtype='object')

#### 'post_id', 'Category', 'Page', 'Post URL', 'Date Published', 'Post Type', 'Rating', 'Debate', 'share_count', 'reaction_count', 'comment_count', 'Title', and 'account_id' columns are not necessary for textual prediction, removing un-necessary columns

In [13]:
df = df_marge.drop(['post_id', 'Category', 'Page', 'Post URL', 'Date Published','Post Type', 'Rating', 'Debate', 'share_count', 'reaction_count','comment_count', 'Title', 'account_id'], axis = 1)

In [14]:
df.isnull().sum()

Content    0
class      0
dtype: int64

#### Shuffling dataframe randomly

In [15]:
df = df.sample(frac = 1)

In [16]:
df.head()

,Content,class
88,FBI Director James Comey warned Tuesday that t...,1
14,A suspicious device found in a trash can near ...,1
39,NEWARK — A Donald Trump aide who worked for Go...,1
30,North Carolina Gov. Pat McCrory said Friday th...,1
23,North Carolina Republicans are lamenting the u...,1


In [17]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [18]:
df.columns

Index(['Content', 'class'], dtype='object')

In [19]:
df.head()

,Content,class
0,FBI Director James Comey warned Tuesday that t...,1
1,A suspicious device found in a trash can near ...,1
2,NEWARK — A Donald Trump aide who worked for Go...,1
3,North Carolina Gov. Pat McCrory said Friday th...,1
4,North Carolina Republicans are lamenting the u...,1


#### Defining Dataframe Cleaning Function: Creating a function to convert the text in lowercase, remove the extra space, special chr., ulr and links.
        - Converting to lowercase
        - removing special characters
        - removing extra space
        - removing URL and hyperlinks
        - Removing extra line space


In [20]:
def dataFilter(newsContent):
    newsContent = newsContent.lower()
    newsContent = re.sub("\\W"," ",newsContent) 
    newsContent = re.sub('<.*?>+', '', newsContent)
    newsContent = re.sub('\n', '', newsContent)
    newsContent = re.sub('https?://\S+|www\.\S+', '', newsContent)
    newsContent = re.sub('\[.*?\]', '', newsContent)
    newsContent = re.sub('[%s]' % re.escape(string.punctuation), '', newsContent)
    newsContent = re.sub('\w*\d\w*', '', newsContent)
    return newsContent

In [21]:
df["text"] = df["Content"].apply(dataFilter)

#### Defining x as dependent and y as independent variable

In [22]:
x = df["text"]
y = df["class"]

#### Dividing dataset into two sets
        1. Training Set
        2. Testing Set

In [23]:
trainSet_x, testSet_x, trainSet_y, testSet_y = train_test_split(x, y, test_size=0.25)

#### Vectorizing Textual content

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
v = TfidfVectorizer()
xv_train = v.fit_transform(trainSet_x)
xv_test = v.transform(testSet_x)

### 1. Naive Bayes Classifier

In [26]:
#defining a classifier
gaussianNB = GaussianNB()
gaussianNB.fit(xv_train.toarray(), trainSet_y)

GaussianNB()

In [27]:
pred_gussianNB= gaussianNB.predict(xv_test.toarray())

In [28]:
gaussianNB.score(xv_test.toarray(), testSet_y)

0.7857142857142857

In [29]:
print(classification_report(testSet_y, pred_gussianNB))

              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.78      1.00      0.88        21

    accuracy                           0.79        28
   macro avg       0.89      0.57      0.56        28
weighted avg       0.83      0.79      0.72        28



### 2. Logistic Regression

In [30]:
LoRe = LogisticRegression()
LoRe.fit(xv_train,trainSet_y)

LogisticRegression()

In [31]:
pred_LoRe=LoRe.predict(xv_test)

In [32]:
LoRe.score(xv_test, testSet_y)

0.75

In [33]:
print(classification_report(testSet_y, pred_LoRe))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.75      1.00      0.86        21

    accuracy                           0.75        28
   macro avg       0.38      0.50      0.43        28
weighted avg       0.56      0.75      0.64        28



c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 3. Decision Tree Classifier

In [34]:
DeTr = DecisionTreeClassifier()
DeTr.fit(xv_train, trainSet_y)

DecisionTreeClassifier()

In [35]:
pred_DeTr = DeTr.predict(xv_test)

In [36]:
DeTr.score(xv_test, testSet_y)

0.8214285714285714

In [37]:
print(classification_report(testSet_y, pred_DeTr))

              precision    recall  f1-score   support

           0       0.75      0.43      0.55         7
           1       0.83      0.95      0.89        21

    accuracy                           0.82        28
   macro avg       0.79      0.69      0.72        28
weighted avg       0.81      0.82      0.80        28



### 4. Gradient Boosting Classifier

In [38]:
GrBoCl = GradientBoostingClassifier(random_state=0)
GrBoCl.fit(xv_train, trainSet_y)

GradientBoostingClassifier(random_state=0)

In [39]:
pred_GrBoCl = GrBoCl.predict(xv_test)

In [40]:
GrBoCl.score(xv_test, testSet_y)

0.8571428571428571

In [41]:
print(classification_report(testSet_y, pred_GrBoCl))

              precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       0.84      1.00      0.91        21

    accuracy                           0.86        28
   macro avg       0.92      0.71      0.76        28
weighted avg       0.88      0.86      0.83        28



### 5. Random Forest Classifier

In [42]:
RaFoCl = RandomForestClassifier(random_state=0)
RaFoCl.fit(xv_train, trainSet_y)

RandomForestClassifier(random_state=0)

In [43]:
pred_RaFoCl = RaFoCl.predict(xv_test)

In [44]:
RaFoCl.score(xv_test, testSet_y)

0.75

In [45]:
print(classification_report(testSet_y, pred_RaFoCl))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.75      1.00      0.86        21

    accuracy                           0.75        28
   macro avg       0.38      0.50      0.43        28
weighted avg       0.56      0.75      0.64        28



c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ananda\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Manual Testing


In [46]:
def op_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pnda.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(dataFilter) 
    new_x_test = new_def_test["text"]
    new_xv_test = v.transform(new_x_test)
    pred_GNB = gaussianNB.predict(new_xv_test.toarray())
    pred_LR = LoRe.predict(new_xv_test)
    pred_DT = DeTr.predict(new_xv_test)
    pred_GBC = GrBoCl.predict(new_xv_test)
    pred_RFC = RaFoCl.predict(new_xv_test)

    return print("\n\nGaussian Naive Bayes Prediction: {} \nLogistic Regression Prediction: {} \nDecision Tree Prediction: {} \nGradient Boosting Prediction: {} \nRandom Forest Prediction: {}".format(op_lable(pred_GNB[0]),
                                                                                                            op_lable(pred_LR[0]), 
                                                                                                              op_lable(pred_DT[0]), 
                                                                                                              op_lable(pred_GBC[0]), 
                                                                                                              op_lable(pred_RFC[0])))

In [47]:
news = str(input())
manual_testing(news)



Gaussian Naive Bayes Prediction: Not A Fake News 
Logistic Regression Prediction: Not A Fake News 
Decision Tree Prediction: Not A Fake News 
Gradient Boosting Prediction: Not A Fake News 
Random Forest Prediction: Not A Fake News
